# 1. QR Decomposition (```linalg.qr```)

$$
A = QR
$$


$A \in \mathbb{R}^{m \times n}$,  $Q \in \mathbb{R}^{m \times m}$,  $R \in \mathbb{R}^{m \times n}$ 

* $Q$: Orthogonal matrix
* $QR$: representation of $A$ in terms of orthonormal basis
* Actual computational method used - Not Gram-Schmidt but **Householder** method (O(n$^2$)) For more on this, refer to [QR algorithm](https://github.com/JKang918/Linear-Algebra-with-Python/blob/main/11.%20Overview%20-%20QR%20Algorithm.ipynb)

```python
Q, R = lingalg.qr(A, mode="full") # default # not recommended 
Q, R = lingalg.qr(A, mode="economic") # recommended
```

* corresponding Lapack function: ```geqrf```, ```orgqr```, ```ungqr```

```mode="full"``` returns $Q \in \mathbb{R}^{m \times m}$,  $R \in \mathbb{R}^{m \times n}$, per QR Decomposition.

$$
\begin{bmatrix}
| & | &        & | \\
\mathbf{a}_1 & \mathbf{a}_2 & \cdots & \mathbf{a}_n \\
| & | &        & |
\end{bmatrix}
=
\begin{bmatrix}
| & | &        & | \\
\mathbf{q}_1 & \mathbf{q}_2 & \cdots & \mathbf{q}_m \\
| & | &        & |
\end{bmatrix}
\begin{bmatrix}
\mathbf{a}_1 \cdot \mathbf{q}_1 & \mathbf{a}_2 \cdot \mathbf{q}_1 & \cdots & \mathbf{a}_n \cdot \mathbf{q}_1 \\
0 & \mathbf{a}_2 \cdot \mathbf{q}_2 & \cdots & \mathbf{a}_n \cdot \mathbf{q}_2 \\
\vdots & \ddots & \ddots & \vdots \\
0 & \cdots & 0 & \mathbf{a}_n \cdot \mathbf{q}_n \\
0 & \cdots & 0 &  0 \\
\vdots & \ddots & \ddots & \vdots \\
0 & \cdots & 0 &  0 \\
\end{bmatrix}
$$


```mode="economic"``` returns $Q \in \mathbb{R}^{m \times n}$,  $R \in \mathbb{R}^{n \times n}$ (m $\geq$ n)


$$
\begin{bmatrix}
| & | &        & | \\
\mathbf{a}_1 & \mathbf{a}_2 & \cdots & \mathbf{a}_n \\
| & | &        & |
\end{bmatrix}
=
\begin{bmatrix}
| & | &        & | \\
\mathbf{q}_1 & \mathbf{q}_2 & \cdots & \mathbf{q}_n \\
| & | &        & |
\end{bmatrix}
\begin{bmatrix}
\mathbf{a}_1 \cdot \mathbf{q}_1 & \mathbf{a}_2 \cdot \mathbf{q}_1 & \cdots & \mathbf{a}_n \cdot \mathbf{q}_1 \\
0 & \mathbf{a}_2 \cdot \mathbf{q}_2 & \cdots & \mathbf{a}_n \cdot \mathbf{q}_2 \\
\vdots & \ddots & \ddots & \vdots \\
0 & \cdots & 0 & \mathbf{a}_n \cdot \mathbf{q}_n \\
\end{bmatrix}
$$


Example. Economic case

$$
A = 
\begin{bmatrix}
1 & 0 & 0\\
1 & 1 & 0\\
1 & 1 & 1\\
1 & 1 & 1\\
\end{bmatrix}, \quad
Q = 
\begin{bmatrix}
1/2 & -3/\sqrt{12} &  0\\
1/2 &  1/\sqrt{12} & -2/\sqrt{6}\\
1/2 &  1/\sqrt{12} &  1/\sqrt{6}\\
1/2 &  1/\sqrt{12} &  1/\sqrt{6}\\
\end{bmatrix} \approx
\begin{bmatrix}
0.5 & -0.8660 & 0\\
0.5 &  0.2887 & -0.8165/\sqrt{6}\\
0.5 &  0.2887 &  0.4082\\
0.5 &  0.2887 &  0.4082\\
\end{bmatrix} 
, \quad
R = 
\begin{bmatrix}
2 & 3/2 &  1\\
0 &  3/\sqrt{12} & 2/\sqrt{12}\\
0 &  0 &  2/\sqrt{6}\\
\end{bmatrix} \approx
\begin{bmatrix}
2 & 1.5 &  1\\
0 &  0.8660 & 0.5774\\
0 &  0 &  0.8165\\
\end{bmatrix}
$$

In [1]:
import numpy as np
from scipy import linalg

In [2]:
""" constructing A """

A = np.tri(4, 3, k=0, dtype=np.float64)

In [3]:
""" QR Decomposition """

Q, R = linalg.qr(A, mode="economic")

In [4]:
print(Q)
print(R)

[[-0.5         0.8660254   0.        ]
 [-0.5        -0.28867513  0.81649658]
 [-0.5        -0.28867513 -0.40824829]
 [-0.5        -0.28867513 -0.40824829]]
[[-2.         -1.5        -1.        ]
 [ 0.         -0.8660254  -0.57735027]
 [ 0.          0.         -0.81649658]]


In [5]:
""" A = QR """
print(Q @ R)
print(np.allclose(A, Q@R))

[[ 1.00000000e+00  8.69063787e-17 -1.34358683e-16]
 [ 1.00000000e+00  1.00000000e+00  1.61842956e-16]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00]]
True


# 2. (Crude Replication of) One-Step Approach for QR Algorithm 

$$
A = Q_1 R_1 \rightarrow A_1 = R_1Q_1
$$
$$
A_1 = Q_2 R_2 \rightarrow A_2 = R_2Q_2
$$
$$
\vdots
$$
$$
A_{k-1} = Q_k R_k \rightarrow A_k = R_kQ_k
$$

Then, $A_k$ converges to an upper triangular matrix (Schur form). All $A_i$'s are similar. The diagonal entries of $A_k$ are eigenvalues.

$$
A =
\begin{bmatrix}
1 & 3 & 3\\
-3 & -5 & -3 \\
3 & 3 & 1 
\end{bmatrix}
$$

Eigenvalues: $\lambda_1 = 1$, $\lambda_2 = -2$, $\lambda_3 = -2$

In [6]:
A = np.array([
    [1, 3, 3],
    [-3, -5, -3],
    [3, 3, 1]
])

In [7]:
""" linalg.eig """
eigvals = linalg.eig(A, right=False)
print(eigvals)

[ 1.+0.j -2.+0.j -2.+0.j]


In [8]:
""" Custom QR algorithm (educational purpose) """

for _ in range(100):
    Q, R = linalg.qr(A, mode="economic")
    A = R@Q

print(A) # upper triangular
np.diag(A) # eigenvalues

[[-2.00000000e+00  1.01865655e-15  7.34846923e+00]
 [-1.85353854e-16 -2.00000000e+00  4.24264069e+00]
 [-9.66153348e-31  5.57808895e-31  1.00000000e+00]]


array([-2., -2.,  1.])

# 3. Hessenberg Reduction (reference: QR Algorithm, Two-step approach)

$$
A = UHU^\top \qquad A = UHU^H
$$

$H$: upper Hessenberg, $U$: orthogonal or unitary matrix

$$
H=
\begin{bmatrix}
x & x & x & x & x \\
x & x & x & x & x \\
 & x & x & x & x \\
 &  & x & x & x \\
 & & & x & x
\end{bmatrix}
$$

```python
H = linalg.hessenberg(A, calc_q=False) # default
H, U = linalg.hessenberg(A, calc_q=True)
```

* corresponding Lapack function: ```gehrd```, ```orghr```, ```unghr```

For more detail, refer to [QR algorithm](https://github.com/JKang918/Linear-Algebra-with-Python/blob/main/11.%20Overview%20-%20QR%20Algorithm.ipynb), Two step approach

In [9]:
A = np.random.rand(10, 10)
# H = linalg.hessenberg(A)
H, U = linalg.hessenberg(A, calc_q=True)
# print(A)
# print(H)

In [10]:
""" Sanity Check """

print(np.allclose(A, U@H@U.T))

True


## Practice

Make 10 x 10 matrix with eigenvalues = 1, 2, ... , 10
- Make D matrix with diagonal entries = 1, 2, ..., 10
- Make P matrix with random entries in [-1, 1]
- A = PDP^-1

Calculate eigenvalues using ```linalg.eig```

Use Crude One-step approache to calculate eigenvalues

In [11]:
""" Construct D """
diag = np.arange(10) + 1
D = np.diag(diag, k=0)
print(D.shape)

(10, 10)


In [12]:
""" Construct P """
P = 2*np.random.rand(10, 10) - 1
print(P.shape)

(10, 10)


In [13]:
""" Construct A """
A = P@D@linalg.inv(P) 

In [14]:
""" linalg.eig """
eigvals = linalg.eig(A, right=False)
print(np.sort(np.real(eigvals)))

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


In [15]:
""" One-step """

for _ in range(1000):
    Q, R = linalg.qr(A, mode="economic")
    A = R@Q
print(np.sort(np.diag(A))) # eigenvalues

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
